In [1]:
import pandas as pd
from dash import Dash,html,dcc, dash_table
#import dash_design_kit as ddk
import plotly.express as px
import pandas as pd
from datetime import date, timedelta
import datetime
import numpy as np
from dash.dependencies import Input, Output, State
import time
#import boto3
import requests

In [2]:
# Reading in the Superstore sample dataset from data world
df = pd.read_excel('https://query.data.world/s/37bge7u6amq2pcluvt4tu73j7s7sxr')

In [3]:
# Verifying the dataset
df

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country/Region,City,...,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,1,CA-2019-152156,2019-11-08,2019-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420.0,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136
1,2,CA-2019-152156,2019-11-08,2019-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420.0,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820
2,3,CA-2019-138688,2019-06-12,2019-06-16,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,90036.0,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00,6.8714
3,4,US-2018-108966,2018-10-11,2018-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311.0,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310
4,5,US-2018-108966,2018-10-11,2018-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311.0,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5164
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9989,9990,CA-2017-110422,2017-01-21,2017-01-23,Second Class,TB-21400,Tom Boeckenhauer,Consumer,United States,Miami,...,33180.0,South,FUR-FU-10001889,Furniture,Furnishings,Ultra Door Pull Handle,25.2480,3,0.20,4.1028
9990,9991,CA-2020-121258,2020-02-26,2020-03-03,Standard Class,DB-13060,Dave Brooks,Consumer,United States,Costa Mesa,...,92627.0,West,FUR-FU-10000747,Furniture,Furnishings,Tenex B1-RE Series Chair Mats for Low Pile Car...,91.9600,2,0.00,15.6332
9991,9992,CA-2020-121258,2020-02-26,2020-03-03,Standard Class,DB-13060,Dave Brooks,Consumer,United States,Costa Mesa,...,92627.0,West,TEC-PH-10003645,Technology,Phones,Aastra 57i VoIP phone,258.5760,2,0.20,19.3932
9992,9993,CA-2020-121258,2020-02-26,2020-03-03,Standard Class,DB-13060,Dave Brooks,Consumer,United States,Costa Mesa,...,92627.0,West,OFF-PA-10004041,Office Supplies,Paper,"It's Hot Message Books with Stickers, 2 3/4"" x 5""",29.6000,4,0.00,13.3200


In [4]:
# Looking at the datatypes
df.dtypes

Row ID                     int64
Order ID                  object
Order Date        datetime64[ns]
Ship Date         datetime64[ns]
Ship Mode                 object
Customer ID               object
Customer Name             object
Segment                   object
Country/Region            object
City                      object
State                     object
Postal Code              float64
Region                    object
Product ID                object
Category                  object
Sub-Category              object
Product Name              object
Sales                    float64
Quantity                   int64
Discount                 float64
Profit                   float64
dtype: object

In [5]:
# Looking at values in the category column
df['Category'].value_counts()

Office Supplies    6026
Furniture          2121
Technology         1847
Name: Category, dtype: int64

In [6]:
# Looking at dates
print(df['Order Date'].max())
print(df['Order Date'].min())

2020-12-30 00:00:00
2017-01-03 00:00:00


In [7]:
# Building default tables for each of the categories
office_df = df[df['Category'] == 'Office Supplies']
furniture_df = df[df['Category'] == 'Furniture']
technology_df = df[df['Category'] == 'Technology']

In [8]:
# Looking at the shape of each of the datasets
print(office_df.shape)
print(furniture_df.shape)
print(technology_df.shape)

(6026, 21)
(2121, 21)
(1847, 21)


In [9]:
# Creating product lists
office_product_list = office_df['Product Name'].unique()
furniture_product_list = furniture_df['Product Name'].unique()
technology_product_list = technology_df['Product Name'].unique()
print(len(office_product_list))
print(len(furniture_product_list))
print(len(technology_product_list))
print(office_product_list)
print(furniture_product_list)
print(technology_product_list)

1058
379
412
['Self-Adhesive Address Labels for Typewriters by Universal'
 "Eldon Fold 'N Roll Cart System" 'Newell 322' ... 'Grip Seal Envelopes'
 'Eureka Hand Vacuum, Bagless'
 'Eldon Jumbo ProFile Portable File Boxes Graphite/Black']
['Bush Somerset Collection Bookcase'
 'Hon Deluxe Fabric Upholstered Stacking Chairs, Rounded Back'
 'Bretford CR4500 Series Slim Rectangular Table'
 'Eldon Expressions Wood and Plastic Desk Accessories, Cherry Wood'
 'Chromcraft Rectangular Conference Tables'
 'Global Deluxe Stacking Chair, Gray'
 'Riverside Palais Royal Lawyers Bookcase, Royale Cherry Finish'
 'Howard Miller 13-3/4" Diameter Brushed Chrome Round Wall Clock'
 "Electrix Architect's Clamp-On Swing Arm Lamp, Black"
 'Atlantic Metals Mobile 3-Shelf Bookcases, Custom Colors'
 "Global Fabric Manager's Chair, Dark Gray" 'Longer-Life Soft White Bulbs'
 'Global Leather Task Chair, Black' 'Novimex Turbo Task Chair'
 'Luxo Economy Swing Arm Lamp'
 "Global Value Mid-Back Manager's Chair, Gray"
 "H

In [10]:
# Setting calendar default in date picker
# Adapting code found here:  https://datascientyst.com/get-todays-date-pandas/#:~:text=To%20get%20today's%20date%20as,()%20and%20pass%20parameter%20%2D%20today%20.
first_show_date = pd.to_datetime('today').strftime("%Y-%m-%d")
first_show_date

'2022-05-14'

In [11]:
print('Running the app...')
# run the app
app = Dash (__name__)
server = app.server

Running the app...


In [12]:
if __name__ == '__main__':
    app.run_server(debug=True)

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


Traceback (most recent call last):
  File "/Users/jstephens/miniconda3/lib/python3.9/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/Users/jstephens/miniconda3/lib/python3.9/site-packages/traitlets/config/application.py", line 845, in launch_instance
    app.initialize(argv)
  File "/Users/jstephens/miniconda3/lib/python3.9/site-packages/traitlets/config/application.py", line 88, in inner
    return method(app, *args, **kwargs)
  File "/Users/jstephens/miniconda3/lib/python3.9/site-packages/ipykernel/kernelapp.py", line 632, in initialize
    self.init_sockets()
  File "/Users/jstephens/miniconda3/lib/python3.9/site-packages/ipykernel/kernelapp.py", line 282, in init_sockets
    self.shell_port = self._bind_socket(self.shell_socket, self.shell_port)
  File "/Users/jstephens/miniconda3/lib/python3.9/site-packages/ipykernel/kernelapp.py", line 229, in _bind_socket
    return self._try_bind_socket(s, port)
  File "/Users/jstephens/minicond

SystemExit: 1

/Users/jstephens/miniconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3377: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
#exit